<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)  # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)  # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential()  # identity인 경우
        if stride != 1:  # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)  # (핵심) skip connection
        out = F.relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes  # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset

data_transformations = {
    "basic": transforms.Compose([
        transforms.ToTensor()
    ]),
    "crop_flip": transforms.Compose([  # 크롭 + 뒤집기
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ])
}

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["basic"])
train_aug_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["crop_flip"])
augmented_train_data = ConcatDataset((train_dataset, train_aug_dataset))

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transformations["basic"])

train_loader = torch.utils.data.DataLoader(augmented_train_data, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 80:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.1640625
Current benign train loss: 2.304425001144409

Current batch: 100
Current benign train accuracy: 0.2890625
Current benign train loss: 1.8200820684432983

Current batch: 200
Current benign train accuracy: 0.359375
Current benign train loss: 1.6969637870788574

Current batch: 300
Current benign train accuracy: 0.4296875
Current benign train loss: 1.4296765327453613

Current batch: 400
Current benign train accuracy: 0.4765625
Current benign train loss: 1.447129249572754

Current batch: 500
Current benign train accuracy: 0.4453125
Current benign train loss: 1.479734182357788

Current batch: 600
Current benign train accuracy: 0.625
Current benign train loss: 1.2573543787002563

Current batch: 700
Current benign train accuracy: 0.46875
Current benign train loss: 1.4412521123886108

Total benign train accuarcy: 42.725
Total benign train loss: 1232.9048200845718

[ Test epoch: 0 ]

Test accuarcy: 45.58
Test average l


Test accuarcy: 83.7
Test average loss: 0.0050232926979660985
Model Saved!

[ Train epoch: 8 ]

Current batch: 0
Current benign train accuracy: 0.921875
Current benign train loss: 0.23351559042930603

Current batch: 100
Current benign train accuracy: 0.9296875
Current benign train loss: 0.2509252429008484

Current batch: 200
Current benign train accuracy: 0.890625
Current benign train loss: 0.35514572262763977

Current batch: 300
Current benign train accuracy: 0.875
Current benign train loss: 0.3027997314929962

Current batch: 400
Current benign train accuracy: 0.90625
Current benign train loss: 0.33007338643074036

Current batch: 500
Current benign train accuracy: 0.84375
Current benign train loss: 0.4469732642173767

Current batch: 600
Current benign train accuracy: 0.921875
Current benign train loss: 0.31777292490005493

Current batch: 700
Current benign train accuracy: 0.9140625
Current benign train loss: 0.21829570829868317

Total benign train accuarcy: 90.649
Total benign train l


Total benign train accuarcy: 93.361
Total benign train loss: 151.9833426065743

[ Test epoch: 15 ]

Test accuarcy: 86.12
Test average loss: 0.004640080957114696
Model Saved!

[ Train epoch: 16 ]

Current batch: 0
Current benign train accuracy: 0.9609375
Current benign train loss: 0.15903298556804657

Current batch: 100
Current benign train accuracy: 0.9140625
Current benign train loss: 0.19392019510269165

Current batch: 200
Current benign train accuracy: 0.9609375
Current benign train loss: 0.12429305911064148

Current batch: 300
Current benign train accuracy: 0.875
Current benign train loss: 0.2925735414028168

Current batch: 400
Current benign train accuracy: 0.90625
Current benign train loss: 0.25793084502220154

Current batch: 500
Current benign train accuracy: 0.9453125
Current benign train loss: 0.1918829083442688

Current batch: 600
Current benign train accuracy: 0.9140625
Current benign train loss: 0.23082374036312103

Current batch: 700
Current benign train accuracy: 0.9375



Current batch: 700
Current benign train accuracy: 0.9453125
Current benign train loss: 0.12213393300771713

Total benign train accuarcy: 94.233
Total benign train loss: 131.555620405823

[ Test epoch: 23 ]

Test accuarcy: 88.65
Test average loss: 0.0037690911069512365
Model Saved!

[ Train epoch: 24 ]

Current batch: 0
Current benign train accuracy: 0.953125
Current benign train loss: 0.17784374952316284

Current batch: 100
Current benign train accuracy: 0.984375
Current benign train loss: 0.1099541038274765

Current batch: 200
Current benign train accuracy: 0.9453125
Current benign train loss: 0.13758940994739532

Current batch: 300
Current benign train accuracy: 0.953125
Current benign train loss: 0.19032834470272064

Current batch: 400
Current benign train accuracy: 0.9453125
Current benign train loss: 0.2114039957523346

Current batch: 500
Current benign train accuracy: 0.9609375
Current benign train loss: 0.14460468292236328

Current batch: 600
Current benign train accuracy: 0.95


Current batch: 500
Current benign train accuracy: 0.9453125
Current benign train loss: 0.1827104687690735

Current batch: 600
Current benign train accuracy: 0.9453125
Current benign train loss: 0.19655482470989227

Current batch: 700
Current benign train accuracy: 0.96875
Current benign train loss: 0.12459051609039307

Total benign train accuarcy: 94.627
Total benign train loss: 122.23886482045054

[ Test epoch: 31 ]

Test accuarcy: 88.1
Test average loss: 0.003964952147006989
Model Saved!

[ Train epoch: 32 ]

Current batch: 0
Current benign train accuracy: 0.9921875
Current benign train loss: 0.08978492766618729

Current batch: 100
Current benign train accuracy: 0.9453125
Current benign train loss: 0.18231132626533508

Current batch: 200
Current benign train accuracy: 0.9765625
Current benign train loss: 0.09461549669504166

Current batch: 300
Current benign train accuracy: 0.96875
Current benign train loss: 0.1011841893196106

Current batch: 400
Current benign train accuracy: 0.906

...


Current batch: 100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.013068336993455887

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.00574216665700078

Current batch: 300
Current benign train accuracy: 0.984375
Current benign train loss: 0.024332702159881592

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.0014068135060369968

Current batch: 500
Current benign train accuracy: 0.9921875
Current benign train loss: 0.02562158927321434

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.0028789399657398462

Current batch: 700
Current benign train accuracy: 0.9921875
Current benign train loss: 0.009169021621346474

Total benign train accuarcy: 99.79
Total benign train loss: 6.084322586422786

[ Test epoch: 63 ]

Test accuarcy: 94.74
Test average loss: 0.0020753164879977705
Model Saved!

[ Train epoch: 64 ]

Current batch: 0
Current benign train accuracy: 1.0
Current


Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0017253815894946456

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0004918966442346573

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.006465895101428032

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.0027862684801220894

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.004072483628988266

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.0038903672248125076

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.001598722767084837

Total benign train accuarcy: 99.854
Total benign train loss: 4.56670559477061

[ Test epoch: 71 ]

Test accuarcy: 94.67
Test average loss: 0.002062641952186823
Model Saved!

[ Train epoch: 72 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 


Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.007414266467094421

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0008664128836244345

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.002155254827812314

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.004855564329773188

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.0023779713083058596

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.005163003224879503

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.004252522252500057

Total benign train accuarcy: 99.926
Total benign train loss: 2.933938218076946

[ Test epoch: 79 ]

Test accuarcy: 94.62
Test average loss: 0.0020814353674650193
Model Saved!

[ Train epoch: 80 ]

Current batch: 0
Current benign train accuracy: 0.9921875
Current benign train 


Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0005701059708371758

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0006976680597290397

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.000668491527903825

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.001148456591181457

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.0011225731577724218

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.0028331063222140074

Current batch: 700
Current benign train accuracy: 0.9921875
Current benign train loss: 0.007778053637593985

Total benign train accuarcy: 99.966
Total benign train loss: 1.630659049435053

[ Test epoch: 87 ]

Test accuarcy: 94.93
Test average loss: 0.0020073743205517532
Model Saved!

[ Train epoch: 88 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign trai


Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0011168584460392594

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0009096242138184607

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0009260995429940522

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.0018205102533102036

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.0004102594102732837

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.007829890586435795

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.0016697399551048875

Total benign train accuarcy: 99.982
Total benign train loss: 1.4060563392704353

[ Test epoch: 95 ]

Test accuarcy: 94.96
Test average loss: 0.001999816407635808
Model Saved!

[ Train epoch: 96 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train lo